<a href="https://colab.research.google.com/github/dk-wei/python-multiprocessing/blob/main/Python_Multiprocessing_Medium_Dave_Wei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import multiprocessing as mp
from tqdm.notebook import tqdm

In [2]:
nprocs = mp.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 2


In [3]:
int_list = [3, 4, 5, 6, 5, 4, 3, 2, 1, 3, 4, 4, 3, 2, 4, 6, 2, 1, 32, 4, 55, 3, 2, 1, 4, 6, 78, 8, 5, 6, 0, 9, 8, 6, 3, 5, 6, 7, 7, 8, 8, 4, 2]

In [4]:
def split(iter, n):  
   k, m = divmod(len(iter), n)
   split_data = [iter[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]
   split_data_order_number = [[i, v] for i, v in enumerate(split_data)]
   return split_data_order_number
sub_list = split(int_list, 6)

In [5]:
import pprint

pprint.pprint(list(sub_list))

[[0, [3, 4, 5, 6, 5, 4, 3, 2]],
 [1, [1, 3, 4, 4, 3, 2, 4]],
 [2, [6, 2, 1, 32, 4, 55, 3]],
 [3, [2, 1, 4, 6, 78, 8, 5]],
 [4, [6, 0, 9, 8, 6, 3, 5]],
 [5, [6, 7, 7, 8, 8, 4, 2]]]


In [6]:
%%time
import multiprocessing as mp
# initial Queue which will be used to save our output
# you can initiate as many Queue as you want, its up to you
qout1 = mp.Queue()
qout2 = mp.Queue()

def func(x, q1, q2):
   '''
   define our function
   x: our sub list input
   q1, q2, q3: container to save our result
   '''
   index = x[0]   # index of sublist
   value = x[1]   # content of each sublist
   res1 = []
   print(f'Job {index} starting\n')
   for i in tqdm(value):
      # here we are using calcualting square value as example, you can replace it with any computation you want
      res1.append(i**2)
      # in multiprocessing process，we use q.put() rather than return to save our output
   q1.put(res1)
   q2.put(index)
   print(f'Job {index} finishing\n')

CPU times: user 2.79 ms, sys: 1.17 ms, total: 3.96 ms
Wall time: 8.14 ms


In [7]:
processes = [mp.Process(target=func, args=(sub, qout1, qout2)) for sub in sub_list]

In [8]:
%%time
for p in processes:
   p.daemon = True
   p.start()
# the computation only get triggered when we run this q.get() method
unsorted_result = [[qout1.get(), qout2.get()] for p in processes]
# we can concatenate and sort result by using the index
result = sum([t[0] for t in sorted(unsorted_result, key=lambda x: x[1])], [])
for p in processes:
   # p.join tells the process to wait until all jobs finished then exit, effectively cleaning up the pool.
   p.join()
   # p.close terminate the pool and tells the process not to accept any new job. 
   p.close()
print('All task is done!')

Job 1 starting
Job 0 starting


Job 4 starting
Job 3 starting
Job 2 starting


Job 5 starting





Job 0 finishing






Job 2 finishing
Job 5 finishing
Job 1 finishing

Job 4 finishing




Job 3 finishing
All task is done!
CPU times: user 103 ms, sys: 43.7 ms, total: 146 ms
Wall time: 383 ms


In [10]:
result == [i**2 for i in int_list]

True